In [12]:
import json
from glob import glob

output_path = "/home0/murakamih/work/Watch-BBox-In-Out/output"

def read_json(json_path: str) -> list:
    with open(json_path, 'r') as f:
        _data = json.load(f)
    return _data

for dataset in glob(output_path+"/*"):
    for version in glob(dataset+"/*"):
        for json_path in glob(version + "/*.json"):
            data = read_json(json_path)
            # print(data)
            for ped_token, ann in data.items():
                try:
                    if ann['looking'] != 'Object':
                        continue
                    if ann['gto_category'] == 'vehicle.car':
                        # print(data)
                except TypeError as e:
                    print(e)
                    print(data)
            # break
        # break
    # break

{'9d098effc0424029bb6585849d818386': {'bbox': [1252.0251064238696, 468.1979136162275, 1464.6981725918154, 747.3310650596493], 'looking': 'Object', 'gto_bbox': [985.0960233163494, 450.35172260702666, 1600.0, 900.0], 'gto_center': [1292.5480116581748, 675.1758613035133], 'gto_category': 'vehicle.car'}}
{'9b91c31a93da44f1b09fbdb0bbaddae4': {'bbox': [998.655014293729, 442.49355809249033, 1165.7576013730852, 770.8932886226743], 'looking': 'Object', 'gto_bbox': [949.349505149002, 384.8436627896735, 1600.0, 708.4592693322975], 'gto_center': [1274.6747525745009, 546.6514660609855], 'gto_category': 'vehicle.car'}, 'b79365da484d44cbb64264ed44599e00': {'bbox': [1059.0783410783956, 443.76431395990306, 1197.9746749168348, 736.7968194109338], 'looking': 'Object', 'gto_bbox': [601.9118326366608, 396.82728002184535, 1104.9286217090134, 861.6030896629742], 'gto_center': [853.4202271728371, 629.2151848424098], 'gto_category': 'vehicle.car'}}
{'9b91c31a93da44f1b09fbdb0bbaddae4': {'bbox': [998.65501429372

TypeError: list indices must be integers or slices, not str